<a href="https://colab.research.google.com/github/lhfp98/git-bootcamp/blob/main/Gibson_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Install tabula-py
!pip install tabula-py

# Import the read_pdf function from tabula
from tabula import read_pdf

# URL to the PDF file containing the tabular data
URL = "/gibson/2019 GIBSON GAS - WEEK (48).pdf"

# Use tabula to read the PDF file from the given URL
tabular_data = read_pdf(URL, pages='all', lattice=True)  # For well-defined grids
tabular_data = read_pdf(URL, pages='all', multiple_tables=True)



In [15]:
tabular_data[4]

,Mts,Unnamed: 0,Unnamed: 1,Unnamed: 2,US $/pmt,vs Previous week
0,"44,000",LPG,AG,Japan,$71.000,Down $5.071
1,"6,500",BTD/PPL,Brazil,USG-Caribs,140-150,Firming
2,"6,500",C2/BTD,Brazil,CJK,290-300,Firming
3,"6,500",Ethylene,USG,N.W.E,165-175,Firming
4,"6,500",Ethylene,N.W.E.,CJK,280-290,Firming
5,"6,500",Ethylene,MEG,SE Asia,100-110,Firming
6,"6,500",Ethylene,MEG,CJK,145-155,Firming
7,"6,500",Butadiene,N.W.E,CJK,270-280,Firming
8,"6,500",Propylene,USG,N.W.E,140-150,Firming
9,"1,600",Propylene,Yosu,E.China,54,Unchanged


In [12]:
import tabula
import pandas as pd
import os

def extract_tables(pdf_path):
    # Attempt to extract tables using tabula
    try:
        # Using stream mode based on your table layout; adjust pages as needed
        tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, stream=True)
        return tables
    except Exception as e:
        print(f"Failed to extract tables from {pdf_path}: {str(e)}")
        return None

def validate_tables(tables, expected_columns):
    # Check if any of the tables match the expected structure
    valid_tables = []
    for table in tables:
        if all(col in table.columns for col in expected_columns):
            valid_tables.append(table)
    return valid_tables

# Specify your PDF directory and expected columns based on your sample images
pdf_directory = '/gibson'
expected_columns = ['Mts', 'US $/pmt', 'vs Previous week']

# Process each PDF in the directory
results = {}
for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        full_path = os.path.join(pdf_directory, pdf_file)
        tables = extract_tables(full_path)
        if tables:
            valid_tables = validate_tables(tables, expected_columns)
            if valid_tables:
                results[pdf_file] = valid_tables
            else:
                results[pdf_file] = "No valid tables found"
        else:
            results[pdf_file] = "No tables extracted"

# Output the results
for key, value in results.items():
    print(f"File: {key}, Result: {value}")


Nov 20, 2024 9:07:47 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 20, 2024 9:07:47 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 20, 2024 9:07:47 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 20, 2024 9:07:48 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Nov 20, 2024 9:08:27 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



File: 2019 GIBSON GAS - WEEK (51).pdf, Result: [       Mts Unnamed: 0 Unnamed: 1   Unnamed: 2 US $/pmt vs Previous week
0   44,000        LPG         AG        Japan  $68.143      Down $0.500
1    6,500    BTD/PPL     Brazil   USG-Caribs  140-150        Unchanged
2    6,500     C2/BTD     Brazil          CJK  290-300        Unchanged
3    6,500   Ethylene        USG        N.W.E  165-175        Unchanged
4    6,500   Ethylene     N.W.E.          CJK  280-290        Unchanged
5    6,500   Ethylene        MEG      SE Asia  100-110        Unchanged
6    6,500   Ethylene        MEG          CJK  145-155        Unchanged
7    6,500  Butadiene      N.W.E          CJK  270-280        Unchanged
8    6,500  Propylene        USG        N.W.E  140-150        Unchanged
9    1,600  Propylene       Yosu      E.China       54        Unchanged
10   1,500  Propylene      S.E.A      E.China      162        Unchanged
11   4,000     Butane      EC UK      Morocco       64          Down $1
12   1,800     B

In [25]:
import tabula
import pandas as pd
import os
from datetime import datetime
import re

import re
from datetime import datetime

def clean_ordinal(date_string):
    # Remove ordinal suffixes such as 'st', 'nd', 'rd', 'th'
    return re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_string)

def extract_date_from_pdf(pdf_path):
    area = [10, 350, 100, 600]  # New adjusted area for capturing the date at the top right
    try:
        tables = tabula.read_pdf(pdf_path, area=area, pages=1, multiple_tables=True, pandas_options={'header': None})
        if tables:
            date_text_df = tables[0]
            if not date_text_df.empty:
                date_string = date_text_df.iloc[0, 0]
                cleaned_date_string = clean_ordinal(date_string)
                date_object = datetime.strptime(cleaned_date_string, '%d %B %Y')
                return date_object.strftime('%Y-%m-%d')
            else:
                return "No data in extracted area"
        else:
            return "No tables extracted"
    except Exception as e:
        print(f"Error extracting date from {pdf_path}: {e}")
        return None

# Example usage of the function
pdf_path = "/gibson/2019 GIBSON GAS - WEEK (48).pdf"
extracted_date = extract_date_from_pdf(pdf_path)
print(f"Extracted Date: {extracted_date}")



def extract_and_validate_tables(pdf_path, expected_columns):
    tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, stream=True)
    valid_tables = []
    for table in tables:
        if all(col in table.columns for col in expected_columns):
            valid_tables.append(table)
    return valid_tables

def process_pdfs(pdf_directory, expected_columns):
    results = {}
    for pdf_file in os.listdir(pdf_directory):
        if pdf_file.endswith('.pdf'):
            full_path = os.path.join(pdf_directory, pdf_file)
            date_extracted = extract_date_from_pdf(full_path)
            if date_extracted:
                tables = extract_and_validate_tables(full_path, expected_columns)
                for table in tables:
                    table['Report Date'] = date_extracted
                results[pdf_file] = tables
            else:
                results[pdf_file] = "Failed to extract date"
    return results

# Example usage
pdf_directory = '/gibson'
expected_columns = ['Mts', 'US $/pmt', 'vs Previous week']
processed_data = process_pdfs(pdf_directory, expected_columns)

# Output the results
for key, value in processed_data.items():
    print(f"File: {key}, Result: {value}")


Error extracting date from /gibson/2019 GIBSON GAS - WEEK (48).pdf: time data 'Week 48 |  28 November 2019' does not match format '%d %B %Y'
Extracted Date: None
Error extracting date from /gibson/2019 GIBSON GAS - WEEK (51).pdf: time data 'Week 51 |  20 December 2019' does not match format '%d %B %Y'
Error extracting date from /gibson/2019 GIBSON GAS - WEEK (48).pdf: time data 'Week 48 |  28 November 2019' does not match format '%d %B %Y'
Error extracting date from /gibson/2019 GIBSON GAS - WEEK (50).pdf: time data 'Week 50 |  12 December 2019' does not match format '%d %B %Y'
Error extracting date from /gibson/2019 GIBSON GAS - WEEK (49).pdf: time data 'Week 49 |  5 December 2019' does not match format '%d %B %Y'
File: 2019 GIBSON GAS - WEEK (51).pdf, Result: Failed to extract date
File: 2019 GIBSON GAS - WEEK (48).pdf, Result: Failed to extract date
File: 2019 GIBSON GAS - WEEK (50).pdf, Result: Failed to extract date
File: 2019 GIBSON GAS - WEEK (49).pdf, Result: Failed to extract 

## Working code 2019

In [32]:

import tabula
import pandas as pd
import os
from datetime import datetime
import re

def clean_ordinal(date_string):
    # Remove ordinal suffixes such as 'st', 'nd', 'rd', 'th'
    return re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_string)

def clean_date_string(date_string):
    # Regular expression to find the date portion
    match = re.search(r'\d{1,2}(st|nd|rd|th)?\s+[a-zA-Z]+\s+\d{4}', date_string)
    if match:
        # Apply clean_ordinal to the matched date string
        return clean_ordinal(match.group(0))
    else:
        return None

def extract_date_from_pdf(pdf_path):
    area = [10, 350, 100, 600]  # Adjusted area from prior suggestion
    try:
        tables = tabula.read_pdf(pdf_path, area=area, pages=1, multiple_tables=True, pandas_options={'header': None})
        if tables:
            date_text_df = tables[0]
            if not date_text_df.empty:
                raw_date_string = date_text_df.iloc[0, 0]
                cleaned_date_string = clean_date_string(raw_date_string)
                if cleaned_date_string:
                    date_object = datetime.strptime(cleaned_date_string, '%d %B %Y')
                    return date_object.strftime('%Y-%m-%d')
                else:
                    return "Date format not recognized"
            else:
                return "No data in extracted area"
        else:
            return "No tables extracted"
    except Exception as e:
        print(f"Error extracting date from {pdf_path}: {e}")
        return None

# Example usage of the function
pdf_path = "/gibson"
extracted_date = extract_date_from_pdf(pdf_path)
print(f"Extracted Date: {extracted_date}")



def extract_and_validate_tables(pdf_path, expected_columns):
    tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, stream=True)
    valid_tables = []
    for table in tables:
        if all(col in table.columns for col in expected_columns):
            valid_tables.append(table)
    return valid_tables

def process_pdfs(pdf_directory, expected_columns, output_csv_path):
    all_tables = []  # List to store all DataFrames
    for pdf_file in os.listdir(pdf_directory):
        if pdf_file.endswith('.pdf'):
            full_path = os.path.join(pdf_directory, pdf_file)
            date_extracted = extract_date_from_pdf(full_path)
            if date_extracted:
                tables = extract_and_validate_tables(full_path, expected_columns)
                for table in tables:
                    table['Report Date'] = date_extracted
                    all_tables.append(table)  # Append each table to the list
            else:
                print(f"Failed to extract date from {pdf_file}")

    # Concatenate all DataFrames into one
    if all_tables:
        combined_df = pd.concat(all_tables, ignore_index=True)
        # Save the DataFrame to CSV
        combined_df.to_csv(output_csv_path, index=False)
        print(f"Data successfully written to {output_csv_path}")
        return combined_df
    else:
        print("No data to display.")
        return None

# Example usage
pdf_directory = '/gibson'
expected_columns = ['Mts', 'US $/pmt', 'vs Previous week']
output_csv_path = 'combined_data_2019.csv'  # Specify the path to save the CSV file
combined_data = process_pdfs(pdf_directory, expected_columns, output_csv_path)

# The combined DataFrame is also returned if you need to use it further in the script
if combined_data is not None:
    print(combined_data)
else:
    print("No data to display.")




#Extracted Date: 2019-01-03
#Error extracting date from /gibson/2019 GIBSON GAS - WEEK (48).pdf: time data '48 |  28 November 2019' does not match format '%d %B %Y'
#Failed to extract date from 2019 GIBSON GAS - WEEK (48).pdf
#Error extracting date from /gibson/2019 GIBSON GAS - WEEK (50).pdf: time data '50 |  12 December 2019' does not match format '%d %B %Y'
#Failed to extract date from 2019 GIBSON GAS - WEEK (50).pdf
#Error extracting date from /gibson/2019 GIBSON GAS - WEEK (51).pdf: time data '51 |  20 December 2019' does not match format '%d %B %Y'
#Failed to extract date from 2019 GIBSON GAS - WEEK (51).pdf
#Error extracting date from /gibson/2019 GIBSON GAS - WEEK (46).pdf: time data '46 |  14 November 2019' does not match format '%d %B %Y'
#Failed to extract date from 2019 GIBSON GAS - WEEK (46).pdf
#Error extracting date from /gibson/2019 GIBSON GAS - WEEK (47).pdf: time data '47 |  21 November 2019' does not match format '%d %B %Y'
#Failed to extract date from 2019 GIBSON GAS - WEEK (47).pdf
#Error extracting date from /gibson/2019 GIBSON GAS - WEEK (49).pdf: time data '49 |  5 December 2019' does not match format '%d %B %Y'
#Failed to extract date from 2019 GIBSON GAS - WEEK (49).pdf

ERROR:tabula.backend:Error from tabula-java:
Error: /gibson (Is a directory)




Error extracting date from /gibson: Command '['java', '-Dfile.encoding=UTF8', '-jar', '/usr/local/lib/python3.10/dist-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar', '--pages', '1', '--area', '10,350,100,600', '--format', 'JSON', '/gibson']' returned non-zero exit status 1.
Extracted Date: None


Nov 20, 2024 9:36:53 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Nov 20, 2024 9:41:23 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Data successfully written to combined_data_2019.csv
        Mts Unnamed: 0 Unnamed: 1   Unnamed: 2 US $/pmt vs Previous week  \
0    44,000        LPG         AG        Japan  $62.143        Up $8.714   
1     6,500    BTD/PPL     Brazil   USG-Caribs  150-160        Unchanged   
2     6,500     C2/BTD     Brazil          CJK  290-300        Unchanged   
3     6,500   Ethylene        USG        N.W.E  165-175        Unchanged   
4     6,500   Ethylene     N.W.E.          CJK  290-300        Unchanged   
..      ...        ...        ...          ...      ...              ...   
811   4,000     Butane      EC UK      Morocco       65          Firming   
812   1,800     Butane       Tees          ARA       48        Unchanged   
813   1,800     Butane       Tees       Lisbon      107        Unchanged   
814   1,600    Propane       Tees        Brest       73        Unchanged   
815   1,800        LPG    S.China  Philippines       62        Unchanged   

    Report Date  
0    2019-04-18  

In [33]:
combined_data


,Mts,Unnamed: 0,Unnamed: 1,Unnamed: 2,US $/pmt,vs Previous week,Report Date
0,"44,000",LPG,AG,Japan,$62.143,Up $8.714,2019-04-18
1,"6,500",BTD/PPL,Brazil,USG-Caribs,150-160,Unchanged,2019-04-18
2,"6,500",C2/BTD,Brazil,CJK,290-300,Unchanged,2019-04-18
3,"6,500",Ethylene,USG,N.W.E,165-175,Unchanged,2019-04-18
4,"6,500",Ethylene,N.W.E.,CJK,290-300,Unchanged,2019-04-18
...,...,...,...,...,...,...,...
811,"4,000",Butane,EC UK,Morocco,65,Firming,2019-04-05
812,"1,800",Butane,Tees,ARA,48,Unchanged,2019-04-05
813,"1,800",Butane,Tees,Lisbon,107,Unchanged,2019-04-05
814,"1,600",Propane,Tees,Brest,73,Unchanged,2019-04-05


In [34]:
combined_data['Report Date'] = pd.to_datetime(combined_data['Report Date'], errors='coerce')
# 'errors='coerce'' will turn any non-convertible values into NaT (Not a Time), which helps avoid crashes due to format issues.

unique_dates = combined_data['Report Date'].dropna().unique()
unique_dates_df = pd.DataFrame(unique_dates, columns=['Unique Dates'])
sorted_dates = unique_dates_df.sort_values(by='Unique Dates')
print(sorted_dates)


   Unique Dates
10   2019-01-03
18   2019-01-10
5    2019-01-17
3    2019-01-24
17   2019-01-31
4    2019-02-07
49   2019-02-14
16   2019-02-21
38   2019-02-28
40   2019-03-07
31   2019-03-15
48   2019-03-22
45   2019-03-28
50   2019-04-05
33   2019-04-11
0    2019-04-18
39   2019-04-25
6    2019-05-02
36   2019-05-09
7    2019-05-16
41   2019-05-24
44   2019-05-31
27   2019-06-07
47   2019-06-13
35   2019-06-20
42   2019-06-27
34   2019-07-04
9    2019-07-11
12   2019-07-18
30   2019-07-25
21   2019-08-01
43   2019-08-09
32   2019-08-16
26   2019-08-23
37   2019-08-29
25   2019-09-06
11   2019-09-12
14   2019-09-19
15   2019-09-26
46   2019-10-04
29   2019-10-10
8    2019-10-17
13   2019-10-24
24   2019-11-01
23   2019-11-07
22   2019-11-14
20   2019-11-21
2    2019-11-28
28   2019-12-05
19   2019-12-12
1    2019-12-20
